<p align="center"><img width="50%" src="https://www.modelshare.ai/img/ModelShareLogo_COLOR.abf79305.png" /></p>


---




<p align="center"><h1 align="center">Quick Start: Flower Image Classification Tutorial using Keras</h1> 


## **Create Model Playground ML Project Space: Submit, Track, and Deploy ML Model(s) in about 5 mins**
> ### (1) Set Up Environment 
> ### (2) Create a Model Playground
> ### (3) Submit, Track, and Deploy ML Model(s)
> ### (4) Improve Models and Analyze Model Performance
> ### (5) Maintain Your Model Playground

## **(1) Set Up Environment**

In order to deploy a Model Share AI Model Playground, you will need a credentials text file. 

Generating your credentials file requires two sets of information: 
1. Your Model Share AI username and password (create them [HERE](https://www.modelshare.org/login)). 
2. Your AWS (Amazon Web Services) access keys (follow the tutorial [HERE](https://docs.aws.amazon.com/general/latest/gr/aws-sec-cred-types.html)). 

You only need to generate your credentials file once. After running the configure function below, save the outputted file for all your future Model Playground deployments and competition submissions. 

*Note: Handle your credentials file with the same level of security you handle your passwords. Do not share your file with anyone, send via email, or upload to Github. Model Share AI does not store your credentials.*


In [ ]:
#install aimodelshare library
! pip install aimodelshare --upgrade

In [ ]:
# Generate credentials file
from aimodelshare.aws import configure_credentials 
configure_credentials()

Modelshare.ai Username:··········
Modelshare.ai Password:··········
AWS_ACCESS_KEY_ID:··········
AWS_SECRET_ACCESS_KEY:··········
AWS_REGION:··········
Configuration successful. New credentials file saved as 'credentials.txt'


In [ ]:
# Set credentials 
from aimodelshare.aws import set_credentials
set_credentials(credential_file="credentials.txt", type="deploy_model")

Modelshare.ai login credentials set successfully.
AWS credentials set successfully.


In [ ]:
# # Download flower image data and and pretrained Keras models
from aimodelshare.data_sharing.download_data import import_quickstart_data
keras_model, y_train_labels = import_quickstart_data("flowers")
keras_model_2, y_test_labels = import_quickstart_data("flowers", "competition")


Data downloaded successfully.

Preparing downloaded files for use...

Success! Your Quick Start materials have been downloaded. 
You are now ready to run the tutorial.

Data downloaded successfully.

Preparing downloaded files for use...

Success! Your Quick Start materials have been downloaded. 
You are now ready to run the tutorial.


## **(2) Create a Model Playground**

### **Create a Model Playground**


> ### Each Model Playground is a unique ML project space with lots of useful functionality...
- Use playground.submit_model() to track ML models in experiments and/or competition leaderboards
- Save, version, and reuse ML models each time they are submitted
- Use playground.compare_models() to compare model architectures with git like colorized model diffs
- Use playground.deploy_model() to deploy models instantly into live REST APIs and/or web-apps.  

*  *First, you need to intantiate a ModelPlayground() object locally. Please specify the data type of your learning task (e.g., image, tabular, etc), whether the model is a classification or a regression model, and whether your playground should be public or private. If you choose "private" only approved team members can interact your model playground*  
*  *Second, you can to create a playground page on the modelshare.ai website. You need to submit evaluation data that submitted models can be evaluated against.*  



In [ ]:
# Instantiate Model Playground object
from aimodelshare.playground import ModelPlayground
myplayground=ModelPlayground(input_type="image", 
                             task_type="classification", 
                             private=False)

# Create Model Playground Page on modelshare.ai website
myplayground.create(eval_data=y_test_labels)

Creating your prediction API. (This process may take several minutes.)

[=====================================] Progress: 100% - Complete!                                            

Success! Your Model Playground was created in 67 seconds. 
 Playground Url: "https://moyz6xuxu9.execute-api.us-east-1.amazonaws.com/prod/m"

You can now use your Model Playground.

Follow this link to explore your Model Playground's functionality
You can make predictions with the Dashboard and access example code from the Programmatic tab.
https://www.modelshare.ai/detail/model:3522

Check out your Model Playground page for more.


##  **3) Submit, Track, and Deploy Model(s)**


### **Write a Preprocessor Function**


> ###   Preprocessing functions play a crucial role in preparing data for your model by converting it into the specific format that is needed to generate predictions.  

*  *Preprocessor functions should always be named "preprocessor".*
*  *You can use any Python library in a preprocessor function, but all libraries should be imported inside your preprocessor function.*  
*  *For image prediction models users should minimally include function inputs for an image filepath and values to reshape the image height and width.*  



In [ ]:
# Here is a pre-designed preprocessor, but you could also build your own to prepare the data differently
def preprocessor(image_filepath, shape=(192, 192)):
        """
        This function preprocesses reads in images, resizes them to a fixed shape and
        min/max transforms them before converting feature values to float32 numeric values
        required by onnx files.
        
        params:
            image_filepath
                full filepath of a particular image
                      
        returns:
            X
                numpy array of preprocessed image data
        """
           
        import cv2
        import numpy as np

        "Resize a color image and min/max transform the image"
        img = cv2.imread(image_filepath) # Read in image from filepath.
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 reads in images in order of blue green and red, we reverse the order for ML.
        img = cv2.resize(img, shape) # Change height and width of image.
        img = img / 255.0 # Min-max transform.


        # Resize all the images...
        X = np.array(img)
        X = np.expand_dims(X, axis=0) # Expand dims to add "1" to object shape [1, h, w, channels] for keras model.
        X = np.array(X, dtype=np.float32) # Final shape for onnx runtime.
        return X

### **Train Your Machine Learning Model**


> ### Model Share supports Scikit-Learn, Tensorflow,  Keras, and Pytorch models out of the box.  

*  *Train your model using your favorite ML library. Here, we have already loaded a Keras model in the quickstart function above.*  
*  *Use your model to make predictions on the test set.*  




In [ ]:
# Check the model architecture.
keras_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 192, 192, 64)      256       
                                                                 
 dense_3 (Dense)             (None, 192, 192, 64)      4160      
                                                                 
 dense_4 (Dense)             (None, 192, 192, 64)      4160      
                                                                 
 flatten_1 (Flatten)         (None, 2359296)           0         
                                                                 
 dense_5 (Dense)             (None, 5)                 11796485  
                                                                 
Total params: 11,805,061
Trainable params: 11,805,061
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Preprocess X_test image data to generate predictions from models 
import numpy as np

# Generate filenames
file_names = [('flower_competition_data/test_images/' + str(i) + '.jpg') for i in range(1, 735)]

# Apply preprocessor to image data
preprocessed_image_data = [preprocessor(x) for x in file_names]

# Create single X_test array from preprocessed images
X_test = np.vstack(preprocessed_image_data) 

# One-hot encode y_train labels (y_train.columns used to generate prediction labels below)
import pandas as pd
y_train = pd.get_dummies(y_train_labels)

In [ ]:
# Generate predicted y values
prediction_column_index=keras_model.predict(X_test).argmax(axis=1)

# Extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

23/23 [==============================] - 28s 1s/step


### **Submit a Model to your Model Playground**


> ### You can submit models to your Model Playground page in order to evaluate them, extract model metadata, compare them to other models, and turn them into REST API endpoints.  

*  *Pass your model to the submit model function. You can either pass a model object, an ONNX object, or a filepath to an ONNX file.*  
*  *Pass your preprocessor to ensure that your model can process new data.*
*  *Pass your predictions to be evaluated against your playground evaluation data.*  




In [ ]:
# Submit Model to Experiment Leaderboard
# (Option: set submission_type="competition" to submit to competition leaderboard)

myplayground.submit_model(model=keras_model,
                          preprocessor=preprocessor,
                          prediction_submission=prediction_labels)


Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 
Your model has been submitted to experiment as model version 1.

Visit your Model Playground Page for more.
https://www.modelshare.ai/detail/model:3522


### **Deploy a Model to a Prediction REST API and Web-app**


> ### Update the prediction API behind your Model Playground with a new model and verify the model performance metrics in your Model Playground.

Use playground.deploy_model() with the following key arguments:
* *model_version: Simply refer to the model by its leaderboard version number.*
* *example_data: deploy_model builds a web-app for you.  It needs some example data for end users to download to try your web-app.  For image data provide the path to a folder with some example images.* 
*  *y_train: For classification problems deploy_model needs well ordered y_training data labels.  Provide a python list of y_train labels or a one hot encoded Pandas y_training data frame, so your web-app and prediction api return the correct prediction labels*

In [ ]:
# Deploy model by version number
myplayground.deploy_model(model_version=1, 
                          example_data="quickstart_materials/example_data", 
                          y_train=y_train)

Runtime model & preprocessor for api: https://moyz6xuxu9.execute-api.us-east-1.amazonaws.com/prod/m updated to model version 1.

Model metrics are now updated and verified for this model playground.

Visit your Model Playground Page for more.
https://www.modelshare.ai/detail/model:3522


## **(4) Improve Models and Analyze Model Performance**

### **Set Submission Credentials**

> ### Authorized users can submit new models after setting credentials using modelshare.ai username/password.  

*  *This only needs to be done if you want to submit models in a new session.*  
*  *skip this step if you have already set your deployment credentials.*  




In [ ]:
# example url from deployed playground: apiurl= "https://123456.execute-api.us-east-1.amazonaws.com/prod/m
apiurl=myplayground.playground_url 

set_credentials(apiurl=apiurl)

Modelshare.ai Username:··········
Modelshare.ai Password:··········
Modelshare.ai login credentials set successfully.


### **Submit More Models to your Model Playground**


> ### You can submit models to your Model Playground page in order to evaluate them, extract model metadata, compare them to other models, and turn them into REST API endpoints.  

*  *Pass your model to the submit model function. You can either pass a model object, an ONNX object, or a filepath to an ONNX file.*  
*  *Pass your preprocessor to ensure that your model can process new data.*
*  *Pass your predictions to be evaluated against your playground evaluation data.*  


In [ ]:
# Have a look at architecture for model 2
keras_model_2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 192, 192, 32)      416       
                                                                 
 conv2d_1 (Conv2D)           (None, 192, 192, 32)      4128      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 96, 96, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 96, 96, 16)        2064      
                                                                 
 conv2d_3 (Conv2D)           (None, 96, 96, 16)        1040      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 48, 48, 16)       0         
 2D)                                                    

In [ ]:
# Submit Model 2

# Generate predicted y values (Model 2)
prediction_column_index=keras_model_2.predict(X_test).argmax(axis=1)

# extract correct prediction labels (Model 2)
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 2 to Experiment Leaderboard
myplayground.submit_model(model = keras_model_2,
                          preprocessor=preprocessor,
                          prediction_submission=prediction_labels)

23/23 [==============================] - 19s 851ms/step

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 
Your model has been submitted to experiment as model version 2.

Visit your Model Playground Page for more.
https://www.modelshare.ai/detail/model:3522


### **Check Experiment Leaderboard**


> ### Leaderboards allow you to compare model architectures ranked by predictive performance.  

*  *Query leaderboard data for models submitted to your Model Playground page.* 
*  *You can either stylize the leaderboard or use pandas to run your own analyses to explore which model architectures perform well.*  




In [ ]:
# Check experiment leaderboard
data = myplayground.get_leaderboard()
myplayground.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,model_type,depth,num_params,flatten_layers,dropout_layers,conv2d_layers,maxpooling2d_layers,dense_layers,softmax_act,relu_act,loss,optimizer,memory_size,username,version
0,61.58%,60.94%,61.89%,61.32%,keras,Sequential,11,1851153,1,2.000000,4.000000,2.000000,2,1,5,function,RMSprop,7406384,hp2500test,2
1,46.32%,45.21%,48.01%,44.66%,keras,Sequential,5,11805061,1,nan,nan,nan,4,1,3,function,RMSprop,47221344,hp2500test,1


### **Compare Two or More Models**


> ### Model comparisons can help you to identify differences in model architecture at a glance.  

*  *Simply pass the version numbers of the models you would like to compare.* 
*  *For Keras and Pytoch models you can compare architectures through color-coded layer types. For Scikit-Learn models, you can compare hyperparameter settings.*  

In [ ]:
# Compare two or more models
data=myplayground.compare_models([1,2], verbose=1)
myplayground.stylize_compare(data)

,Model_1_Layer,Model_1_Shape,Model_1_Params,Model_2_Layer,Model_2_Shape,Model_2_Params
0,Dense,"[None, 192, 192, 64]",256.000000,Conv2D,"[None, 192, 192, 32]",416
1,Dense,"[None, 192, 192, 64]",4160.000000,Conv2D,"[None, 192, 192, 32]",4128
2,Dense,"[None, 192, 192, 64]",4160.000000,MaxPooling2D,"[None, 96, 96, 32]",0
3,Flatten,"[None, 2359296]",0.000000,Conv2D,"[None, 96, 96, 16]",2064
4,Dense,"[None, 5]",11796485.000000,Conv2D,"[None, 96, 96, 16]",1040
5,None,None,nan,MaxPooling2D,"[None, 48, 48, 16]",0
6,None,None,nan,Dropout,"[None, 48, 48, 16]",0
7,None,None,nan,Flatten,"[None, 36864]",0
8,None,None,nan,Dense,"[None, 50]",1843250
9,None,None,nan,Dropout,"[None, 50]",0


### **Check the Structure of Evaluation Data**

> ### Your team needs to understand the format of the evaluation data, to ensure that their submissions can be processed.  

*  *Make sure class labels, data types, and dimensions of predictions correspond to submitted evaluation data.*
*  *This is important to generate evaluation metrics for your submission.*


In [ ]:
# Check structure of evaluation data
myplayground.inspect_eval_data()

{'ytest_example': ['daisy', 'sunflowers', 'roses', 'dandelion', 'roses'],
 'y_length': 734,
 'class_labels': ['dandelion', 'tulips', 'daisy', 'roses', 'sunflowers'],
 'class_balance': {'daisy': 126,
  'sunflowers': 140,
  'roses': 128,
  'dandelion': 180,
  'tulips': 160},
 'label_dtypes': {"<class 'str'>": 734}}

## **(5) Maintaining Your Model Playground**

### **Update the Runtime Model (the model that is used for predictions in your REST API/Web-App**


> ### You can update the runtime model anytime by picking one of the previously submitted models.  

* *Simply refer to the model by its leaderboard version number.*
* *You can choos a model from the Competition or Experiment leaderboard by specifying the submission_type argument.*

In [ ]:
# Update runtime model
myplayground.update_runtime_model(model_version=2)

Runtime model & preprocessor for api: https://moyz6xuxu9.execute-api.us-east-1.amazonaws.com/prod/m updated to model version 2.

Model metrics are now updated and verified for this model playground.

Visit your Model Playground Page for more.
https://www.modelshare.ai/detail/model:3522


### **Delete Deployment**


> ### You can delete the entire Model Playground, including the REST API, web dashboard, competition, and all submitted models.  

* *This is final - there is no way to recover your Model Playground if you delete it.*
* *We have commented the line for safety reasons.*

In [ ]:
#myplayground.delete_deployment()